In [31]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [32]:
# Load YOLO
net = cv2.dnn.readNet("C:/Users/HP/Downloads/yolov3.weights", "C:/Users/HP/Downloads/yolov3 (1).cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the COCO class labels
with open("C:/Users/HP/Downloads/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Define herbivores and carnivores lists
herbivores = ['antelope', 'bison', 'boar', 'caterpillar', 'cow', 'deer', 'donkey', 'elephant',
              'goat', 'goose', 'giraffe', 'goldfish', 'grasshopper', 'hare', 'horse', 'kangaroo', 
              'koala', 'okapi', 'ox', 'panda', 'pig', 'porcupine', 'reindeer', 'rhinoceros', 
              'sandpiper', 'seahorse', 'sheep', 'sparrow', 'squirrel', 'swan', 'turkey', 
              'turtle', 'wombat', 'zebra']
carnivores = ['badger', 'bat', 'bear', 'bee', 'beetle', 'cat', 'chimpanzee', 'cockroach', 
              'coyote', 'crab', 'crow', 'dog', 'dolphin', 'dragonfly', 'duck', 'eagle', 
              'flamingo', 'fly', 'fox', 'gorilla', 'hamster', 'hedgehog', 'hornbill', 
              'hummingbird', 'hyena', 'jellyfish', 'ladybugs', 'leopard', 'lion', 'lizard', 
              'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'orangutan', 'otter', 'owl', 
              'oyster', 'parrot', 'pelecaniformes', 'penguin', 'pigeon', 'possum', 'raccoon', 
              'rat', 'seal', 'shark', 'snake', 'squid', 'starfish', 'tiger', 'whale', 'wolf', 
              'woodpecker']


In [33]:
def detect_animals(image_path):
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    animal_counts = {'herbivores': 0, 'carnivores': 0}
    detected_animals = []

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            detected_animals.append(label)
            if label in herbivores:
                animal_counts['herbivores'] += 1
            elif label in carnivores:
                animal_counts['carnivores'] += 1
            color = (0, 255, 0) if label in herbivores else (0, 0, 255)
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, label, (x, y + 30), font, 3, color, 3)

    print("Detected Animals:")
    for animal in detected_animals:
        print(animal)

    print(f"Herbivores: {animal_counts['herbivores']}, Carnivores: {animal_counts['carnivores']}")
    
    return image, animal_counts

In [34]:
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        result_image, animal_counts = detect_animals(file_path)
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        result_image_pil = Image.fromarray(result_image)
        
        # Resize image to fit within 800x600 pixels
        max_width = 800
        max_height = 600
        width_ratio = max_width / result_image_pil.width
        height_ratio = max_height / result_image_pil.height
        resize_ratio = min(width_ratio, height_ratio)
        
        new_width = int(result_image_pil.width * resize_ratio)
        new_height = int(result_image_pil.height * resize_ratio)
        
        resized_image_pil = result_image_pil.resize((new_width, new_height))
        
        display_image(resized_image_pil)
        result_text.set(f"Herbivores: {animal_counts['herbivores']}, Carnivores: {animal_counts['carnivores']}")

def display_image(img):
    img_tk = ImageTk.PhotoImage(img)
    panel.config(image=img_tk)
    panel.image = img_tk

# Initialize Tkinter window
root = tk.Tk()
root.title("Animal Detector")

result_text = tk.StringVar()
tk.Label(root, textvariable=result_text).pack()

upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack()

panel = tk.Label(root)
panel.pack()

root.mainloop()

Detected Animals:
Herbivores: 0, Carnivores: 0
Detected Animals:
zebra
Herbivores: 1, Carnivores: 0
Detected Animals:
Herbivores: 0, Carnivores: 0
Detected Animals:
dog
Herbivores: 0, Carnivores: 1
